# FDI Parte 2: Trasformaciones eometricas, histogramas y mascaras.

En esta parte del curso es más preferible empezar a usar librería con funciones muchos más especificas y optimizadas para acelerar los procesos de construcción de código, así que comenzaremos a usar la librería openCV. OpenCV(Open Source Computer Vision Library) es una librería de código abierto que ha estado siendo optimizada desde hace 20 años y posee más de 2500 funciones útiles para el estudio y modificación de imágenes.

## Transformaciones geometricas:

### Redimensionalización:
Para cambiar el tamaño de una imagen, OpenCV viene con la función cv2.resize(). El tamaño deseado de la imagen final se puede especificar manualmente, o se puede indicar especificando un factor de escala. La función usa diferentes métodos de interpolación, siendo los más usados: cv2.INTER_AREA, para contraer la imagen y cv2.INTER_CUBIC (suave) & cv2.INTER_LINEAR, para acercar la imagen. El método de interpolación utilizado por defecto es cv2.INTER_LINEAR, que  sirve para cualquier cambio de tamaño que se desee realizar. Cualquiera de los métodos mostrados a continuación, pueden ser utilizados para cambiar el tamaño de una imagen:

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
#abrir y mostrar y guardar imagenes
img = cv2.imread(ruta)
cv2.imshow("name window", img)
cv2.imwrite("img.png", img)

In [3]:
# x, y son las magninutes de amplificazion de las dimenciones x e y
def Redimensionalizar_escala(img, x, y):
    res = cv2.resize(img,None,fx = x, fy = y, interpolation = cv2.INTER_CUBIC)

# x, y son las magninutes de las dimenciones x e y
def Redimensionalizar_manual(img, x, y):
    res = cv2.resize(img,(x, y), interpolation = cv2.INTER_CUBIC)

### Traslación
Una traslación es el desplazamiento de la posición de un objeto.
se puede escribir la matriz de transformación M utilizando la función np.float32 de la librería Numpy. Esta matriz luego se pasa como argumento a la función cv2.warpAffine().

In [ ]:
def trasladar(im, x, y):
    alto, ancho = img.shape[:2] 
    M = np.float32([[1,0,x],[0,1,y]])
    dst = cv2.warpAffine(img,M,(ancho,alto))
    return dst

### Rotación
OpenCV permite personalizar la rotación multiplacando por un factor de escala. Por otra parte, también permite cambiar el centro de rotación.

Para encontrar esta matriz de transformación, OpenCV proporciona la función cv2.getRotationMatrix2D. 

In [ ]:
def rotar_img(img, grados, centro_x, centro_y, escala):
    alto, ancho = img.shape[:2] 
    M = cv2.getRotationMatrix2D((centro_x, centro_y),grados, escala)
    dst = cv2.warpAffine(img,M,(ancho,alto))
    return dst

### Transformación Afín
En una transformación afín todas las líneas paralelas en la imagen original seguirán siendo paralelas en la imagen de salida. Para encontrar la matriz de transformación, necesitamos tres puntos de la imagen de entrada y sus ubicaciones correspondientes en la imagen de salida. Luego cv2.getAffineTransform creará una matriz 2×3 que se pasará a cv2.warpAffine.

In [ ]:
#puntos: es un conjunto de 3 puntos en forma de lista ej [[1,2],[2,2],[1,3]].
def transformación_afín(im, puntos_i, puntos_f):
    alto, ancho = img.shape[:2] 
    pts1 = np.float32(puntos_i)
    pts2 = np.float32(puntos_f)
    M = cv2.getAffineTransform(pts1,pts2)
    dst = cv2.warpAffine(img,M,(cols,rows))
    # visualizar usando el siguiente codigo
    #plt.subplot(121),plt.imshow(img),plt.title('Input')
    #plt.subplot(122),plt.imshow(dst),plt.title('Output')
    #plt.show()
    
    return dst

### Transformación de Perspectiva
Para realizar una transformación de perspectiva es necesario especificar una matriz de 3×3. Luego de aplicar este tipo de transformación, las líneas rectas permanecerán rectas. Para generar la matriz de 3×3 es necesario  indicar cuatro puntos sobre la imagen de inicial y los correspondientes  puntos sobre la imagen resultante. Tres de los cuatro puntos, tienen que ser no-colineales. De esta manera la matriz de transformación puede ser generada utilizando la función cv2.getPerspectiveTransform. Luego, para aplicar la transformación, se utiliza cv2.warpPerspective teniendo en cuenta la matriz de 3×3 generada con la función anterior.

In [ ]:
# puntos: conjunto de 4 puntos, tres no pueden ser colineales.
# ej : [[1,2],[2,2],[1,3],[3,3]]
# (son las esquinas de la imagen)
def calcular_distancia(punto_1, punto_2):
    x1, y1 = puntos_1[0], puntos_1[1]
    x2, y2 = puntos_2[0], puntos_2[1]
    
    hip = ((x1-x2)**2 + (y1-y2)**2)**(1/2)
    
    return hip
    
def transformación_de_perspectiva(img, puntos):
    alto, ancho = img.shape[:2] 
    
    pts1 = np.float32(puntos)
    delta_x = calcular_distancia(puntos[0], puntos[1])
    delta_y = calcular_distancia(puntos[0], puntos[2])
    pts2 = np.float32([[0,0],[delta_x,0],[0,delta_y],[delta_x,delta_y]])
    
    M = cv2.getPerspectiveTransform(pts1,pts2)
    dst = cv2.warpPerspective(img,M,(delta_x,delta_y))
    # visualizar usando el siguiente codigo
    # plt.subplot(121),plt.imshow(img),plt.title('Input')
    #plt.subplot(122),plt.imshow(dst),plt.title('Output')
    #plt.show()
    return dst

# Histogramas:

los histogramas son una forma útil de visualizar la distribución de colores por tonos de una imagen y a su vez facilita la aplicación de filtros a esta, como suavizar una imagen o aumentar su contraste.

este codigo permite crear un histograma de los colores rojo, verde y azul de una imagen:

In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def crear_histograma_de_colores(ruta): 
    img = cv2.imread(ruta)
    
    color = ('b','g','r')
    
    # i = pocision del str(b,g,r), c = str(b,g,r)
    for i, c in enumerate(color):
        # parametros
        # cv2.calcHist(images, channels, mask, histSize, ranges[, hist[, accumulate]])
        hist = cv2.calcHist([img], [i], None, [256], [0, 256])
        plt.plot(hist, color = c)
        plt.xlim([0,256])

    plt.show()

In [5]:
# crea un histograma en escala de grices
def crear_histograma_gray(ruta):
    img = cv2.imread(ruta, cv2.IMREAD_GRAYSCALE)
    # parametros
    # cv2.calcHist(images, channels, mask, histSize, ranges[, hist[, accumulate]])
    hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    
    # 
    plt.plot(hist, color='gray' )
    plt.xlabel('intensidad de iluminacion')
    plt.ylabel('cantidad de pixeles')
    plt.show()
    
    return hist

In [ ]:
def ecualizar_imagen(im):
    im = cv2.equalizeHist(im)
    cv2.imshow('Histogramas', im)
    cv2.waitKey(0)
    return im

## Convoluciones

In [ ]:
# equivale a pasar una mascara 
def convolucionar(im, matriz):
    kernel = np.array(matriz)
    # opcional:
    # normalizar la matriz(mascara)
    kernel = kernel/(np.sum(kernel) if np.sum(kernel)!=0 else 1)
    # aplicar filtro
    img_conv = cv2.filter2D(im,-1,kernel)
    return img_conv

### fuentes:
* https://unipython.com/transformaciones-geometricas-de-imagenes-con-opencv/
* http://acodigo.blogspot.com/2017/08/histogramas-opencv-python.html
* https://pythonexamples.org/python-opencv-image-filter-convolution-cv2-filter2d/